## Задание 5.1

Набор данных тут: https://github.com/sismetanin/rureviews, также есть в папке [Data](https://drive.google.com/drive/folders/1YAMe7MiTxA-RSSd8Ex2p-L0Dspe6Gs4L). Те, кто предпочитает работать с английским языком, могут использовать набор данных `sms_spam`.

Применим полученные навыки и решим задачу анализа тональности отзывов. 

Нужно повторить весь пайплайн от сырых текстов до получения обученной модели.

Обязательные шаги предобработки:
1. токенизация
2. приведение к нижнему регистру
3. удаление стоп-слов
4. лемматизация
5. векторизация (с настройкой гиперпараметров)
6. построение модели
7. оценка качества модели

Обязательно использование векторайзеров:
1. мешок n-грамм (диапазон для n подбирайте самостоятельно, запрещено использовать только униграммы).
2. tf-idf ((диапазон для n подбирайте самостоятельно, также нужно подбирать параметры max_df, min_df, max_features)
3. символьные n-граммы (диапазон для n подбирайте самостоятельно)

В качестве классификатора нужно использовать наивный байесовский классификатор. 

Для сравнения векторайзеров между собой используйте precision, recall, f1-score и accuracy. Для этого сформируйте датафрейм, в котором в строках будут разные векторайзеры, а в столбцах разные метрики качества, а в  ячейках будут значения этих метрик для соответсвующих векторайзеров.

In [2]:
!pip install pymorphy2

     |████████████████████████████████| 55 kB 1.9 MB/s 
     |████████████████████████████████| 8.2 MB 9.2 MB/s 


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import pandas as pd
import numpy as np
import math
import nltk 
import string

from sklearn.metrics import * 
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

from pymorphy2 import MorphAnalyzer

from itertools import filterfalse
from itertools import product

In [5]:
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [6]:
data = pd.read_csv("/content/drive/MyDrive/ML/women-clothing-accessories.csv", sep="\t", usecols=[0, 1])

In [7]:
data.head()

,review,sentiment
0,качество плохое пошив ужасный (горловина напер...,negative
1,"Товар отдали другому человеку, я не получила п...",negative
2,"Ужасная синтетика! Тонкая, ничего общего с пре...",negative
3,"товар не пришел, продавец продлил защиту без м...",negative
4,"Кофточка голая синтетика, носить не возможно.",negative


In [8]:
class VecScores(object):
  def __init__(self, vectorizer, scores):
    self.vectorizer = vectorizer;
    self.scores = scores

In [26]:
def get_tokens(data):
  comments = []
  for comment in data:
    comments.append([word.lower() for word in word_tokenize(comment)])
  return comments

def get_lemmas(data):
  morph_analyzer = MorphAnalyzer()
  comments = []
  for comment in data:
    comments.append(" ".join([morph_analyzer.parse(word)[0].normal_form for word in comment]))
  return comments

def get_ngram_range_list(range_):
  return list(filterfalse(lambda val: val[0] > val[1], product(range_, range_)))

def calc_vec_score(vec, x_train_, x_test_, y_train_, y_test_, vec_score_list_):
    x_train_ = vec.fit_transform(x_train_)
    x_test_ = vec.transform(x_test_)
    clf = MultinomialNB()
    clf.fit(x_train_, y_train_)
    pred_y = clf.predict(x_test_)
    scores = classification_report(pred_y, y_test_, output_dict=True)["weighted avg"]
    vec_score_list_.append(VecScores(str(vec), scores))


In [ ]:
new_data = data
new_data.rewiew = get_lemmas(get_tokens(data.review))

In [11]:
x_train, x_test, y_train, y_test = train_test_split(new_data.review, new_data.sentiment, train_size = 0.7)

In [ ]:
vec_score_list = []

for ngram_range in get_ngram_range_list(np.arange(2, 4)):
  for max_df in [0.1, 0.5]:
    for min_df in [0, 0.01]:
      for max_features in [8192, 32768]:
        tfidf_vec = TfidfVectorizer(ngram_range=ngram_range, max_df=max_df, min_df=min_df, max_features=max_features)
        calc_vec_score(tfidf_vec, x_train, x_test, y_train, y_test, vec_score_list)

for ngram_range in get_ngram_range_list(np.arange(2, 6)):
  count_word_vec = CountVectorizer(ngram_range=ngram_range, \
                                   stop_words=stopwords.words("russian"), analyzer="word")
  calc_vec_score(count_word_vec, x_train, x_test, y_train, y_test, vec_score_list)

for ngram_range in get_ngram_range_list(np.arange(2, 6)):
  count_char_vec = CountVectorizer(ngram_range=ngram_range, \
                                   stop_words=stopwords.words("russian"), analyzer="char" )
  calc_vec_score(count_char_vec, x_train, x_test, y_train, y_test, vec_score_list)


In [17]:
pd.options.display.max_colwidth = 85

raw_data = []
for vec in vec_score_list:
  raw_data.append({"vectorizer": vec.vectorizer.split(',\n')[0], "precision": vec.scores["precision"], "recall": vec.scores["recall"], "f1_score": vec.scores['f1-score']})
data_result = pd.DataFrame(raw_data)
data_result.sort_values(by=[ "recall", "precision", "f1_score"  ], ascending=False).head(10)


,vectorizer,precision,recall,f1_score
43,"CountVectorizer(analyzer='char', ngram_range=(5, 5)",0.721975,0.719084,0.717330
42,"CountVectorizer(analyzer='char', ngram_range=(4, 5)",0.717644,0.714492,0.712463
40,"CountVectorizer(analyzer='char', ngram_range=(3, 5)",0.714838,0.711159,0.709045
1,"TfidfVectorizer(max_df=0.1, max_features=32768, min_df=0, ngram_range=(2, 2))",0.714932,0.710603,0.711398
5,"TfidfVectorizer(max_df=0.5, max_features=32768, min_df=0, ngram_range=(2, 2))",0.714932,0.710603,0.711398
37,"CountVectorizer(analyzer='char', ngram_range=(2, 5)",0.712501,0.708455,0.706124
41,"CountVectorizer(analyzer='char', ngram_range=(4, 4)",0.711134,0.707233,0.705189
9,"TfidfVectorizer(max_df=0.1, max_features=32768, min_df=0, ngram_range=(2, 3))",0.712246,0.705789,0.707138
13,"TfidfVectorizer(max_df=0.5, max_features=32768, min_df=0, ngram_range=(2, 3))",0.712246,0.705789,0.707138
39,"CountVectorizer(analyzer='char', ngram_range=(3, 4)",0.706441,0.702196,0.699968


Лучшим векторизатором по метрикам "recall", "precision", "f1_score"  оказался CountVectorizer с символьным анализатором и n-граммами - (5, 5). Однако еще хороший результат дает векторизатор - TfidfVectorizer(max_df=0.1, max_features=32768, min_df=0, ngram_range=(2, 2)).

## Задание 5.2 Регулярные выражения

Регулярные выражения - способ поиска и анализа строк. Например, можно понять, какие даты в наборе строк представлены в формате DD/MM/YYYY, а какие - в других форматах. 

Или бывает, например, что перед работой с текстом, надо почистить его от своеобразного мусора: упоминаний пользователей, url и так далее.

Навык полезный, давайте в нём тоже потренируемся.

Для работы с регулярными выражениями есть библиотека **re**

In [ ]:
import re

В регулярных выражениях, кроме привычных символов-букв, есть специальные символы:
* **?а** - ноль или один символ **а**
* **+а** - один или более символов **а**
* **\*а** - ноль или более символов **а** (не путать с +)
* **.** - любое количество любого символа

Пример:
Выражению \*a?b. соответствуют последовательности a, ab, abc, aa, aac НО НЕ abb!

Рассмотрим подробно несколько наиболее полезных функций:

### findall
возвращает список всех найденных непересекающихся совпадений.

Регулярное выражение **ab+c.**: 
* **a** - просто символ **a**
* **b+** - один или более символов **b**
* **c** - просто символ **c**
* **.** - любой символ


In [ ]:
result = re.findall('ab+c.', 'abcdefghijkabcabcxabc') 
print(result)

['abcd', 'abca']


Вопрос на внимательность: почему нет abcx?

**Задание**: вернуть список первых двух букв каждого слова в строке, состоящей из нескольких слов.

In [ ]:
text = "Carpe diem."
re.findall(r"\b\w{2}", text)

['Ca', 'di']

### split
разделяет строку по заданному шаблону


In [ ]:
result = re.split(',', 'itsy, bitsy, teenie, weenie') 
print(result)

['itsy', ' bitsy', ' teenie', ' weenie']


можно указать максимальное количество разбиений

In [ ]:
result = re.split(',', 'itsy, bitsy, teenie, weenie', maxsplit=2) 
print(result)

['itsy', ' bitsy', ' teenie, weenie']


**Задание**: разбейте строку, состоящую из нескольких предложений, по точкам, но не более чем на 3 предложения.

In [ ]:
text = "And although the space they occupy isn't like normal space, nevertheless they are packed in tightly.\
 Not a cubic inch there but is filled by a claw, a talon, a scale, the tip of a tail, so the effect is like \
 one of those trick drawings and your eyeballs eventually realise that the space between each dragon is, in fact, another dragon.\
 They could put you in mind of a can of sardines, if you thought sardines were huge and scaly and proud and arrogant. "

re.split(r"\.", text, maxsplit=2)

["And although the space they occupy isn't like normal space, nevertheless they are packed in tightly",
 ' Not a cubic inch there but is filled by a claw, a talon, a scale, the tip of a tail, so the effect is like  one of those trick drawings and your eyeballs eventually realise that the space between each dragon is, in fact, another dragon',
 ' They could put you in mind of a can of sardines, if you thought sardines were huge and scaly and proud and arrogant. ']

### sub
ищет шаблон в строке и заменяет все совпадения на указанную подстроку

параметры: (pattern, repl, string)

In [ ]:
result = re.sub('a', 'b', 'abcabc')
print (result)

bbcbbc


**Задание**: напишите регулярное выражение, которое позволит заменить все цифры в строке на "DIG".

In [ ]:
text = "Over 1,000 words in the English language are used today because of William Shakespeare."
re.sub(r"\d", "DIG", text)

'Over DIG,DIGDIGDIG words in the English language are used today because of William Shakespeare.'

**Задание**: напишите  регулярное выражение, которое позволит убрать url из строки.

In [ ]:
text = "Piazza home page - https://piazza.com/home."
re.sub(r"http[s]?://[\w.:?/=]+", "", text)

'Piazza home page - '

### compile
компилирует регулярное выражение в отдельный объект

In [ ]:
# Пример: построение списка всех слов строки:
prog = re.compile('[А-Яа-яё\-]+')
prog.findall("Слова? Да, больше, ещё больше слов! Что-то ещё.")

['Слова', 'Да', 'больше', 'ещё', 'больше', 'слов', 'Что-то', 'ещё']

**Задание**: для выбранной строки постройте список слов, которые длиннее трех символов.

In [ ]:
text = "It’s still magic even if you know how it’s done. – from A Hat Full of Sky"
pattern = re.compile(r"\w{4,}")
pattern.findall(text)

['still', 'magic', 'even', 'know', 'done', 'from', 'Full']

**Задание**: вернуть список доменов (@gmail.com) из списка адресов электронной почты:

```
abc.test@gmail.com, xyz@test.in, test.first@analyticsvidhya.com, first.test@rest.biz
```

In [ ]:
text = "abc.test@gmail.com, xyz@test.in, test.first@analyticsvidhya.com, first.test@rest.biz"
pattern = re.compile(r"@[\w.]+")
pattern.findall(text)

['@gmail.com', '@test.in', '@analyticsvidhya.com', '@rest.biz']